# Create Monthly Project Reports in Smartsheet

This should be run once a week or so, to post project updates, for project situational awareness: how many hours are left, how many are worked.

In [1]:
%load_ext nb_black

import os
import json
import logging
from datetime import datetime, timedelta
from typing import Dict, List

import pandas as pd
import numpy as np
import prefect
from box import Box

import smartsheet

# uses the pretty okay SDK here: https://github.com/ProdPerfect/monday
from monday import MondayClient
from mondaydotcom_utils.formatted_value import FormattedValue, get_col_defs
from mondaydotcom_utils.time_block import TimeBlock
from mondaydotcom_utils.utilities import (
    breakout_record,
    get_items_by_board,
    validate_task_record,
)
from prefect import Flow, Parameter, task, unmapped
from prefect.executors import LocalDaskExecutor, LocalExecutor

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

<IPython.core.display.Javascript object>

In [2]:
TASKS_BOARD_ID = "1883170887"
AGREEMENTS_BOARD_ID = "1882423671"
PROJECTS_BOARD_ID = "1882404316"
ACCOUNTS_BOARD_ID = "1882424009"

PROJECT_TASK_TIME_BOARD_ID = "2398200403"

# don't set this here for development work... use the secrets-<environment>.yaml files instead.
MONDAY_KEY = ""
SMARTSHEET_KEY = ""
environment = "dev"

<IPython.core.display.Javascript object>

In [3]:
if not MONDAY_KEY:
    # key hasn't been passed as a papermill parameter... get it from a file?
    secrets = Box.from_yaml(filename=f"secrets-{environment}.yaml")
    MONDAY_KEY = secrets.apps.monday.API_KEY

if not SMARTSHEET_KEY:
    # key hasn't been passed as a papermill parameter... get it from a file?
    secrets = Box.from_yaml(filename=f"secrets-{environment}.yaml")
    SMARTSHEET_KEY = secrets.apps.smartsheet.API_KEY

<IPython.core.display.Javascript object>

In [4]:
# connect monday client
conn = MondayClient(MONDAY_KEY)

<IPython.core.display.Javascript object>

In [5]:
# connect smartsheet client
ss_client = smartsheet.Smartsheet(SMARTSHEET_KEY)
ss_client.errors_as_exceptions(True)

<IPython.core.display.Javascript object>

In [6]:
users = conn.users.fetch_users()["data"]["users"]
users_df = pd.DataFrame(users).set_index("id")
users_df.head()

,name,email,enabled,teams
id,,,,
25810257,Steve Taylor,stephen.taylor@cuanschutz.edu,True,[]
25815853,Faisal Alquaddoomi,faisal.alquaddoomi@cuanschutz.edu,True,[]
25815860,Vincent Rubinetti,vincent.rubinetti@cuanschutz.edu,True,[]
26327954,Audrey Wen,audrey.wen@cuanschutz.edu,True,[]
27773472,timothy.putman@cuanschutz.edu,timothy.putman@cuanschutz.edu,True,[]


<IPython.core.display.Javascript object>

In [7]:
def breakdown_status(x):
    # use this to break down the status columns
    # TODO move this to mondaydotcom-utils in the formatters

    my_list = []
    json1 = json.loads(x)

    if json1.get("text"):
        my_list.append(json1["text"])
    if json1.get("changed_at"):
        my_list.append(json1["changed_at"])

    return ";".join(my_list)

<IPython.core.display.Javascript object>

In [8]:
def get_status_text(x):
    if x:
        json1 = json.loads(x)

        if json1.get("text"):
            return json1["text"]
    return "None"

<IPython.core.display.Javascript object>

In [9]:
accounts_df = get_items_by_board(conn, ACCOUNTS_BOARD_ID)

accounts_df.rename(
    columns={"monday_id": "account_id", "Title": "Client Name"},
    inplace=True,
)

accounts_df.drop(
    columns=[
        "Contacts",
        "Item ID",
        "Subitems",
        "Notes",
        "Customer Projects",
        "Agreements",
        "Type",
    ],
    inplace=True,
    errors="ignore",
)

# convert the yes-no to True-False
accounts_df["No Bill"] = accounts_df["No Bill"].apply(
    lambda x: bool(json.loads(x)["checked"]) if x else False
)

accounts_df

,account_id,Client Name,No Bill
0,1882439999,HealthAI: Admin & Operations,True
1,1882462147,CU SOM: IT Department,False
2,1882588856,CIDA: Center for Innovative Design & Analysis,False
3,1882681138,HealthAI: Greene Lab,False
4,1882681714,HealthAI: TISLab,False
5,1883644776,HealthAI: Bennett Lab,False
6,1883648098,HealthAI: Hunter Lab,False
7,1883649981,HealthAI: Way Lab,False
8,1907269862,HealthAI: Sean Davis,False
9,2246385174,HealthAI: Dwork Lab,False


<IPython.core.display.Javascript object>

In [10]:
projects_df = get_items_by_board(conn, PROJECTS_BOARD_ID)

projects_df.rename(
    columns={
        "monday_id": "project_id",
    },
    inplace=True,
)

projects_df.drop(
    columns=[
        "Repo Description (mirror)",
        "Project Tasks",
        "Subitems",
        "Etimated Time (Hours) (mirror)",
        "Total Task Time (Hours) (mirror)",
        "Project Contacts",
        "SET Resource",
        "Timeline",
        "Customer Source",
        "Tasks Status (mirror)",
        "Dependency",
        "Date Added",
        "Timeline Days",
        "Item ID",
        "Project Health",
    ],
    inplace=True,
    errors="ignore",
)

projects_df["Project Lifecycle"] = projects_df["Project Lifecycle"].apply(
    get_status_text
)

projects_df = projects_df.explode(["Account"], ignore_index=True)
projects_df

,project_id,Title,Account,Grant Number,Notes,Project Closed Date,Project Lifecycle
0,1882442059,TISLab: Monarch UI (3.0) Redesign,1882681714,213359,Was managed and tracked via github and zenhub;...,None,Open
1,1882712838,Greenelab: lab-website-template and related si...,1882681138,None,Bucket of hours for maintaining Jekyll website...,2022-04-06,Closed
2,1882738595,Greenelab: mygeneset.info,1882681138,None,Collaboration with BioThings organization. Web...,None,Open
3,1882739627,CHAI: Manubot next-gen,1882439999,213269,Pending scoping and schedule; Issue resolution...,None,None
4,1882752029,"HealthAI ""lab"" Portfolio Site",1882439999,None,Exhibition and brochure of our (center softwar...,None,Open
5,1882913862,TISLab: Graph DB Deployer,1882681714,None,Closed,2021-12-16,Closed
6,1888314634,CHAI: Admin Technology Foundation,1882439999,None,None,None,Open
7,1892630899,TISLab: Monarch GCP migration,1882681714,None,Closed,2022-03-30,Closed
8,1957293587,Way: Grant Support,1883649981,None,Closed,None,Closed
9,1969468997,"Greenelab: Biomedical Literature ""Word Lapse"" ...",1882681138,213269,Similar in scope and size to https://greenelab...,None,Open


<IPython.core.display.Javascript object>

In [11]:
# add the account to the projects
projects_df = pd.merge(
    projects_df, accounts_df, how="left", left_on="Account", right_on="account_id"
).drop(columns=["account_id"])
projects_df

,project_id,Title,Account,Grant Number,Notes,Project Closed Date,Project Lifecycle,Client Name,No Bill
0,1882442059,TISLab: Monarch UI (3.0) Redesign,1882681714,213359,Was managed and tracked via github and zenhub;...,None,Open,HealthAI: TISLab,False
1,1882712838,Greenelab: lab-website-template and related si...,1882681138,None,Bucket of hours for maintaining Jekyll website...,2022-04-06,Closed,HealthAI: Greene Lab,False
2,1882738595,Greenelab: mygeneset.info,1882681138,None,Collaboration with BioThings organization. Web...,None,Open,HealthAI: Greene Lab,False
3,1882739627,CHAI: Manubot next-gen,1882439999,213269,Pending scoping and schedule; Issue resolution...,None,None,HealthAI: Admin & Operations,True
4,1882752029,"HealthAI ""lab"" Portfolio Site",1882439999,None,Exhibition and brochure of our (center softwar...,None,Open,HealthAI: Admin & Operations,True
5,1882913862,TISLab: Graph DB Deployer,1882681714,None,Closed,2021-12-16,Closed,HealthAI: TISLab,False
6,1888314634,CHAI: Admin Technology Foundation,1882439999,None,None,None,Open,HealthAI: Admin & Operations,True
7,1892630899,TISLab: Monarch GCP migration,1882681714,None,Closed,2022-03-30,Closed,HealthAI: TISLab,False
8,1957293587,Way: Grant Support,1883649981,None,Closed,None,Closed,HealthAI: Way Lab,False
9,1969468997,"Greenelab: Biomedical Literature ""Word Lapse"" ...",1882681138,213269,Similar in scope and size to https://greenelab...,None,Open,HealthAI: Greene Lab,False


<IPython.core.display.Javascript object>

In [12]:
def breakout_time_sessions(row):
    """
    Break down the Monday.com time structure into something simpler for us.

    This is used with a DataFrame.apply()
    """

    mct = TimeBlock()
    mct.parse(row["Actual Time"])
    return mct.total_duration_hours, mct.time_records

<IPython.core.display.Javascript object>

In [13]:
# only getting not posted tasks
tasks_df = get_items_by_board(conn, TASKS_BOARD_ID)

# Only include Ready tasks
tasks_df = tasks_df.loc[
    ~tasks_df["Integration Message"].str.startswith("Posted", na=False)
]

tasks_df.rename(
    columns={
        "monday_id": "task_id",
    },
    inplace=True,
)

# break the time sessions out
tasks_df[["Total Duration Hours", "Time Sessions"]] = tasks_df.apply(
    breakout_time_sessions, axis=1, result_type="expand"
)

tasks_df["Hours"] = tasks_df["Actual Hours"].fillna(0) + tasks_df[
    "Total Duration Hours"
].fillna(0)
tasks_df["Estimated Hours"] = tasks_df["Timeline Days"].fillna(0) * 8

tasks_df["Status"] = tasks_df["Status"].apply(get_status_text)

tasks_df.drop(
    columns=[
        "Subtasks",
        "Timeline Hours (Estimated) (formula)",
        "Total Actual Hours (formula)",
        "Customer Repos",
        "Project Lifecycle (mirror)",
        "task_id",
        "Dependencies",
        "Issue URL",
        "Pull Request URL",
        "Owner",
        "Actual Hours",
        "Actual Time",
        "Total Duration Hours",
        "Time Sessions",
        "Integration Message",
        "Timeline",
        "Timeline Days",
        "Date Added",
        "Actual Time",
        "Date Completed",
        "Notes",
        "Date Added",
    ],
    inplace=True,
    errors="ignore",
)

# projects should be limited to just one, so this will bring it out of the list
tasks_df = tasks_df.explode(["Customer Project"], ignore_index=True)
tasks_df.head()

,Title,Customer Project,Status,Hours,Estimated Hours
0,automating import of wakatime hours into monda...,2208602434,Done,5.883333,0.0
1,extra node page visualizations (beyond 2.0),2249818009,None,0.000000,80.0
2,"node page associations section, table mode",1882442059,Done,16.000000,160.0
3,home page infographic,2249818009,None,0.000000,80.0
4,tools page infographic,2249818009,None,0.000000,80.0


<IPython.core.display.Javascript object>

Finally merge the tasks and projects together for a final task list.

In [14]:
df = pd.merge(
    tasks_df,
    projects_df,
    how="left",
    left_on="Customer Project",
    right_on="project_id",
)

# We only the wants those we bill for
df = df.loc[~df["No Bill"]]

df.rename(
    columns={
        "Title_x": "Title",
        "Title_y": "Project Title",
        "Hours": "Task Hours",
    },
    inplace=True,
)

df.drop(
    columns=[
        "monday_id",
        "task_end_year",
        "task_end_month",
        "Account",
        "No Bill",
    ],
    inplace=True,
    errors="ignore",
)

df.head()

,Title,Customer Project,Status,Task Hours,Estimated Hours,project_id,Project Title,Grant Number,Notes,Project Closed Date,Project Lifecycle,Client Name
1,extra node page visualizations (beyond 2.0),2249818009,None,0.0,80.0,2249818009,TISLab: Monarch UI (3.0) Extended,None,Pending: Enhancements and extended Monarch UI ...,None,None,HealthAI: TISLab
2,"node page associations section, table mode",1882442059,Done,16.0,160.0,1882442059,TISLab: Monarch UI (3.0) Redesign,213359,Was managed and tracked via github and zenhub;...,None,Open,HealthAI: TISLab
3,home page infographic,2249818009,None,0.0,80.0,2249818009,TISLab: Monarch UI (3.0) Extended,None,Pending: Enhancements and extended Monarch UI ...,None,None,HealthAI: TISLab
4,tools page infographic,2249818009,None,0.0,80.0,2249818009,TISLab: Monarch UI (3.0) Extended,None,Pending: Enhancements and extended Monarch UI ...,None,None,HealthAI: TISLab
5,setup automatic product video gifs,2249818009,None,0.0,40.0,2249818009,TISLab: Monarch UI (3.0) Extended,None,Pending: Enhancements and extended Monarch UI ...,None,None,HealthAI: TISLab


<IPython.core.display.Javascript object>

In [15]:
# Collect tasks that are complete, so estimates no longer matter.
report_done_df = (
    df.loc[df.Status == "Done"]
    .groupby(["Client Name", "Project Title"])
    .agg(
        {
            "Estimated Hours": "sum",
            "Task Hours": "sum",
            "Grant Number": "first",
            "Notes": "first",
            "Project Lifecycle": "first",
            "project_id": "first",
        }
    )
    .reset_index()
)

report_done_df["Estimated Hours"] = 0
report_done_df["Task Status"] = "Done"
report_done_df

,Client Name,Project Title,Estimated Hours,Task Hours,Grant Number,Notes,Project Lifecycle,project_id,Task Status
0,HealthAI: Greene Lab,"Greenelab: Biomedical Literature ""Word Lapse"" ...",0,19.50,213269,Similar in scope and size to https://greenelab...,Open,1969468997,Done
1,HealthAI: TISLab,TISLab: Monarch UI (3.0) Redesign,0,101.85,213359,Was managed and tracked via github and zenhub;...,Open,1882442059,Done
2,HealthAI: TISLab,TISLab: Staffing/Support 2021,0,2.00,None,General non-grant staff support,Open,2303312548,Done


<IPython.core.display.Javascript object>

In [16]:
# collect tasks that are incomplete, subtracting the done time from the estimate
report_undone_df = (
    df.loc[df.Status != "Done"]
    .groupby(["Client Name", "Project Title"])
    .agg(
        {
            "Estimated Hours": "sum",
            "Task Hours": "sum",
            "Grant Number": "first",
            "Notes": "first",
            "Project Lifecycle": "first",
            "project_id": "first",
        }
    )
    .reset_index()
)
report_undone_df["Estimated Hours"] = (
    report_undone_df["Estimated Hours"] - report_undone_df["Task Hours"]
)
report_undone_df["Task Status"] = "Not Done"
report_undone_df

,Client Name,Project Title,Estimated Hours,Task Hours,Grant Number,Notes,Project Lifecycle,project_id,Task Status
0,HealthAI: Greene Lab,"Greenelab: Biomedical Literature ""Word Lapse"" ...",1.0,7.0,213269,Similar in scope and size to https://greenelab...,Open,1969468997,Not Done
1,HealthAI: Greene Lab,Greenelab: Staffing/Support 2021,4.0,4.0,None,General non-grant staff support,Open,2303324267,Not Done
2,HealthAI: Greene Lab,Greenelab: mygeneset.info,72.0,8.0,None,Collaboration with BioThings organization. Web...,Open,1882738595,Not Done
3,HealthAI: TISLab,TISLab: Monarch UI (3.0) Extended,848.0,0.0,None,Pending: Enhancements and extended Monarch UI ...,None,2249818009,Not Done
4,HealthAI: TISLab,TISLab: Monarch UI (3.0) Redesign,8.0,0.0,213359,Was managed and tracked via github and zenhub;...,Open,1882442059,Not Done
5,HealthAI: TISLab,TISLab: Staffing/Support 2021,13.5,2.5,None,General non-grant staff support,Open,2303312548,Not Done
6,HealthAI: Way Lab,Way Lab: Staffing/Support 2021,12.0,4.0,None,General non-grant staff support,Open,2334955423,Not Done


<IPython.core.display.Javascript object>

In [17]:
# shuffle the done and undone together
report_df = (
    pd.concat([report_done_df, report_undone_df])
    .groupby(["Client Name", "Project Title"])
    .agg(
        {
            "Estimated Hours": "sum",
            "Task Hours": "sum",
            "project_id": "first",
            "Grant Number": "first",
            "Notes": "first",
            "Project Lifecycle": "first",
            # "Task Status": lambda x: ";".join(list(x)),
        }
    )
    .reset_index()
)
report_df

,Client Name,Project Title,Estimated Hours,Task Hours,project_id,Grant Number,Notes,Project Lifecycle
0,HealthAI: Greene Lab,"Greenelab: Biomedical Literature ""Word Lapse"" ...",1.0,26.50,1969468997,213269,Similar in scope and size to https://greenelab...,Open
1,HealthAI: Greene Lab,Greenelab: Staffing/Support 2021,4.0,4.00,2303324267,None,General non-grant staff support,Open
2,HealthAI: Greene Lab,Greenelab: mygeneset.info,72.0,8.00,1882738595,None,Collaboration with BioThings organization. Web...,Open
3,HealthAI: TISLab,TISLab: Monarch UI (3.0) Extended,848.0,0.00,2249818009,None,Pending: Enhancements and extended Monarch UI ...,None
4,HealthAI: TISLab,TISLab: Monarch UI (3.0) Redesign,8.0,101.85,1882442059,213359,Was managed and tracked via github and zenhub;...,Open
5,HealthAI: TISLab,TISLab: Staffing/Support 2021,13.5,4.50,2303312548,None,General non-grant staff support,Open
6,HealthAI: Way Lab,Way Lab: Staffing/Support 2021,12.0,4.00,2334955423,None,General non-grant staff support,Open


<IPython.core.display.Javascript object>

Now, Smartsheet's turn?

In [18]:
sheet_name = "SE Project Journal"

search_results = ss_client.Search.search(sheet_name).results

# helpful: https://stackoverflow.com/questions/52065527/python-best-way-to-get-smartsheet-sheet-by-name
sheet_id = next(
    result.object_id for result in search_results if result.object_type == "sheet"
)
sheet = ss_client.Sheets.get_sheet(sheet_id)
sheet_id

4818113414883204

<IPython.core.display.Javascript object>

In [19]:
# break down the cell IDs into a quick lookup box
cell_ids = {}
for column in sheet.columns:
    my_column = column.to_dict()
    cell_ids[my_column["title"]] = my_column["id"]
cell_ids

{'Reported Date': 8072731988125572,
 'Project Title': 754382593648516,
 'Project Lifecycle': 27674127165316,
 'Estimated Hours Remaining': 3006182407333764,
 'Unposted Task Hours': 7092057810462596,
 'Notes': 7509782034704260,
 'Account/Client': 1880282500491140,
 'Grant Proposal #': 6383882127861636}

<IPython.core.display.Javascript object>

Add the records to Smartsheet

In [20]:
right_now = datetime.now().strftime("%Y-%m-%d")

rows = []
for k, v in report_df.to_dict("index").items():

    row = ss_client.models.row.Row()

    row.cells.append(
        {"column_id": cell_ids["Account/Client"], "value": v["Client Name"]}
    )
    if v.get("Grant Number"):
        row.cells.append(
            {"column_id": cell_ids["Grant Proposal #"], "value": v["Grant Number"]}
        )
    #     if v.get("Notes"):
    #         row.cells.append({"column_id": cell_ids["Notes"], "value": v["Notes"]})
    row.cells.append(
        {"column_id": cell_ids["Project Title"], "value": v["Project Title"]}
    )
    row.cells.append({"column_id": cell_ids["Reported Date"], "value": right_now})
    row.cells.append(
        {
            "column_id": cell_ids["Estimated Hours Remaining"],
            "value": v["Estimated Hours"],
        }
    )
    row.cells.append(
        {"column_id": cell_ids["Unposted Task Hours"], "value": v["Task Hours"]}
    )
    row.cells.append(
        {"column_id": cell_ids["Project Lifecycle"], "value": v["Project Lifecycle"]}
    )

    row.to_bottom = True
    rows.append(row)


result = ss_client.Sheets.add_rows(sheet_id, rows)

<IPython.core.display.Javascript object>